In [5]:

import pandas as pd
import xgboost as xgb

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import re
import joblib

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

labelencoder = LabelEncoder()

# import training set & test set
train = pd.read_csv('train.csv')

train = train[['comment_text', 'toxic']]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\githe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\githe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# pre-processing
stop_words = stopwords.words('english')
wnl = WordNetLemmatizer()

def preprocess(text_column):
    new_comment = []
    for comment in text_column:
        text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+",
                 ' ', str(comment).lower()).strip()
        text = [wnl.lemmatize(i) for i in text.split(' ') if i not in stop_words]
        new_comment.append(' '.join(text))
        return new_comment

    train['comment_text']=preprocess(train['comment_text'])

In [3]:
# vectorization
X = train.drop(columns=['toxic'])
y = train.loc[:, 'toxic']


cv = CountVectorizer(binary = True)
cv.fit(train['comment_text'])

X = cv.transform(X['comment_text'])

X

<159571x189775 sparse matrix of type '<class 'numpy.int64'>'
	with 6949691 stored elements in Compressed Sparse Row format>

In [4]:
# Build Model
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

clf = xgb.XGBClassifier(
    use_label_encoder=False,
    colsample_bytree= 0.625,
    gamma= 0.35000000000000003,
    learning_rate= 0.6000000000000001,
    min_child_weight=1.5,
    n_estimators=1327,
    tree_method='gpu_hist',
    subsample=1.0
)
# Training
print('Training...')
clf.fit(X, y)
print('Finished.')


# Predict
# print('Predicting...')
# y_pred = clf.predict(X_test)
# print('Finished.')

# display model performance on training set
## confusion matrix
#conf_matrix = confusion_matrix(y_test, y_pred)
# sns.heatmap(conf_matrix, annot=True, fmt='g')

# plt.show()

## classification report
# print(classification_report(y_test, y_pred, digits=4))

Training...
[08:31:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [ ]:
# Save Model
filename = 'finalized_model.sav'
joblib.dump(clf, filename)
